<a href="https://colab.research.google.com/github/wjdrnqja9/TIL/blob/main/deep_learning/2_DL_(Embedding_%26_Bidirectional_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### ❗️ 본 실습자료는 저작권법에 의거하여 **본 교육 外 배포/게시/공개/공유를 금합니다.**

<br>
<br>

# 0. Install & import libraries 

In [1]:
!pip install konlpy==0.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 3.8 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 453 kB 53.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import konlpy
import sklearn
import tensorflow

print(konlpy.__version__) # 0.5.2
print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0

0.5.2
1.0.2
2.8.2


In [3]:
import pandas as pd
import numpy as np

import re
import os
import sys
import json
import pickle
from collections import Counter

from konlpy.tag import Okt 

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import models, layers, losses, optimizers, metrics, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

<br>
<br>

# 1. Load the data

In [ ]:
# import urllib.request
# import tqdm

# urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

# reviews_df = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
# reviews_df = reviews_df.drop_duplicates('reviews') 
# reviews_df = reviews_df.reset_index(drop=True)
# reviews_df['label'] = reviews_df['ratings'].apply(lambda x : 1 if x > 3 else 0)

# def tokenize_and_stemming(sentence, tokenizer):
#     sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence) 
#     raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming
#     sentence_tokenized = []
#     for token, pos in raw_pos_tagged: # ex) word <- ('서울', 'Noun')
#         # 제거 대상 : ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Suffix", "Determiner"]
#         if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
#             sentence_tokenized.append(token)  
#     return sentence_tokenized

# okt = Okt()
# tokenized_sentences = []
# for sentence in tqdm.tqdm(reviews_df['reviews']):
#     try:
#         tokenized_sentences.append(tokenize_and_stemming(sentence, okt))
#     except:
#         print("Error occured at :", sentence)
#         tokenized_sentences.append([])
# reviews_df['reviews'] = tokenized_sentences

# from google.colab import drive
# drive.mount('/gdrive')

# data_path = '/gdrive/MyDrive/colab_data/temp_data/' 
# # 구글 드라이브 내 "data_path" 경로의 폴더가 존재하지 않을 시 새로이 생성
# if not os.path.exists(data_path): 
#     os.makedirs(data_path)
# reviews_df.to_csv(data_path + 'data_reviews_tokenized.csv', encoding='utf-8', index=False)

# data_path = '/gdrive/MyDrive/colab_data/temp_data/' 
# reviews_df = pd.read_csv(data_path + 'data_reviews_tokenized.csv', encoding='utf-8')
# reviews_df['reviews'] = reviews_df['reviews'].apply(eval)

# train_x, test_x, train_y, test_y = model_selection.train_test_split(reviews_df['reviews'], reviews_df['label'], 
#                                                                     test_size=0.3, 
#                                                                     random_state=42) 

In [4]:
# 앞선 실습 과정에서 구글드라이브에 저장해 둔 전처리를 마친 데이터를 로딩

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
data_path = '/gdrive/MyDrive/colab_data/temp_data/' 

reviews_df = pd.read_csv(data_path + 'data_reviews_tokenized.csv', encoding='utf-8')
reviews_df['reviews'] = reviews_df['reviews'].apply(eval) # eval()의 역할 : "['편하다', '가성', '최고']" (str) -> ['편하다', '가성', '최고'] (list)

reviews_df.head(3)

,ratings,reviews,label
0,5,"[배공, 빠르다]",1
1,2,"[택배, 엉망, 없이, 놔두다]",0
2,5,"[아주, 좋다, 바지, 정말, 좋다, 구매, 하다, 가격, 대박, 이다, 바느질, ...",1


In [6]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(reviews_df['reviews'], reviews_df['label'], 
                                                                    test_size=0.3, 
                                                                    random_state=42) 

# 별도로 stratify 파라미터 값을 지정하지 않아도 자동으로 클래스 비율이 유지됨
# print(train_y.value_counts() / len(train_y))

<br>
<br>

# 2. 텍스트 데이터 전처리 (Tokens -> Sequence numbers -> Sequences with Padding)

### 1) Tokenizing & Tokens to sequence numbers

In [7]:
# 전체 형태소 중 단 1회만 출현했던 형태소의 비중 검토 
# (불필요할 경우 stopwords와 유사한 관점에서 삭제 진행)

tokens = []

for tokens_list in train_x:
    tokens.extend(tokens_list)

tokens_counted = Counter(tokens)


num_all_token = len(tokens_counted) # 중복 제거한 전체 형태소 개수
num_rare_token = 0 # 1회만 출현한 형태소의 개수를 count하기 위한 변수

# ex) '재다': 11641
for token, count in tokens_counted.items():
    if count == 1:
        num_rare_token += 1


print('- 중복 제거한 전체 형태소의 수 :', num_all_token)
print('- 출현 횟수가 1회인 희귀 형태소의 수 :', num_rare_token)

print('- 중복 제거한 전체 형태소 중 희귀 형태소의 비율 :', (num_rare_token / num_all_token) * 100)
print('- 중복 포함한 전체 형태소 중 희귀 형태소 출현 횟수의 비율 :', (num_rare_token / len(tokens)) * 100)


# 단 1회만 출현했던 형태소는 전체 형태소 종류 중 50% 만큼이나 되지만,
# 실제로 형태소들의 출현 빈도를 기반으로 살펴보았을 때에는 그 비중이 1.25% 밖에 되지 않음 
# 1회만 출현했던 형태소들을 제외하고, 나머지 형태소들만 활용해도 큰 지장이 없음 (총 16473개)

- 중복 제거한 전체 형태소의 수 : 33087
- 출현 횟수가 1회인 희귀 형태소의 수 : 16614
- 중복 제거한 전체 형태소 중 희귀 형태소의 비율 : 50.213074621452535
- 중복 포함한 전체 형태소 중 희귀 형태소 출현 횟수의 비율 : 1.2516630830961446


In [8]:
# 어휘 목록의 크기 지정 
# - 전체 token 개수 중 출현 횟수가 1회인 token의 개수를 제거
# - 추후 설명할 [Padding] token & 아래 설명할 [OOV] token 을 고려해 미리 "2" 추가 
# - 추후 [Padding] token 은 token 번호가 0으로 지정되며, [OOV] token 은 token 번호가 1로 지정됨

vocab_size = num_all_token - num_rare_token + 2 # 33087 - 16614 + 2

print('어휘 목록(token vocabulary)의 크기 :', vocab_size)

어휘 목록(token vocabulary)의 크기 : 16475


### * 아래 코드를 통해 만들어낼 결과물인 word_vocab 변수의 모습을 먼저 확인하고 시작합니다.

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>') # tensorflow.keras.preprocessing.text.Tokenizer @ https://keras.io/ko/preprocessing/text/
# "num_words" : 단어 빈도에 따른 사용할 단어 개수의 최대값 (가장 빈번하게 사용되는 num_words 개의 단어만 보존) 
# -> 출현 빈도 내림차순으로 16475 개의 token만 유지 
# "oov_token" : oov_token parameter에 특정한 문자열 값이 지정된 경우, 해당 문자열이 단어 색인(word_index)에 추가되어 texts_to_sequences() 호출 과정에서 어휘 목록 외의 단어를 대체합니다.
# -> 추후 새로운 문자열에 대해 predict 과정을 진행할 때, training data에 포함되어있지 않은 단어(형태소)를 대상으로 매칭되어 sequence 처리 & predict가 가능하도록 해줍니다. (+ 출현 횟수 1회 형태소 처리)

tokenizer.fit_on_texts(train_x) # only "training" data

# reviews_df['reviews']에 포함된 전체 형태소들에 대해 임의의 Sequence number들과 1:1로 매칭시켜 변환
# -> Sequence number는 "0"이 아닌 "1"부터 시작 ("0"은 추후 나올 padding으로 추가될 0을 위해 활용)
train_sequences = tokenizer.texts_to_sequences(train_x) 
test_sequences = tokenizer.texts_to_sequences(test_x) 

# [ 형태소 : Sequence number ]로 이루어진 dict
word_vocab = tokenizer.word_index

In [ ]:
# word_vocab

In [ ]:
# len(word_vocab) # 33088 = num_all_token(33087) + '<OOV>' token

In [10]:
print( len(train_x) )
print( len(train_sequences) )

139935
139935


In [11]:
print(train_x.iloc[7])
print(train_sequences[7])

['사무실', '쓰다', '허리', '안으프', '좋다']
[941, 15, 463, 1, 3]


In [12]:
# number of unique sequence numbers (== 1회만 출현했던 형태소들을 제외한 나머지 형태소들의 개수 (16473개) + '<OOV>' token)

flat_review_sequences = []
for sublist in train_sequences:
    for item in sublist:
        flat_review_sequences.append(item)

print(np.min(flat_review_sequences))
print(np.max(flat_review_sequences))

print(len(set(flat_review_sequences))) 

1
16474
16474


In [13]:
print(word_vocab) # [ 형태소 : Sequence number ]로 이루어진 dict

{'<OOV>': 1, '하다': 2, '좋다': 3, '너무': 4, '배송': 5, '같다': 6, '있다': 7, '자다': 8, '먹다': 9, '재다': 10, '않다': 11, '구매': 12, '없다': 13, '이다': 14, '쓰다': 15, '오다': 16, '사용': 17, '되다': 18, '사다': 19, '보다': 20, '제품': 21, '빠르다': 22, '가격': 23, '그냥': 24, '생각': 25, '받다': 26, '주문': 27, '아니다': 28, '많이': 29, '별로': 30, '사이즈': 31, '들다': 32, 'ㅠㅠ': 33, '만족하다': 34, '맛있다': 35, '포장': 36, '크다': 37, '써다': 38, '편하다': 39, '작다': 40, '그렇다': 41, '맞다': 42, '상품': 43, '안되다': 44, '괜찮다': 45, '저렴하다': 46, '이쁘다': 47, '넘다': 48, '싸다': 49, '냄새': 50, '정말': 51, '부분': 52, '보내다': 53, '시키다': 54, '나다': 55, '버리다': 56, '예쁘다': 57, '하나': 58, '모르다': 59, 'ㅎㅎ': 60, '아주': 61, '진짜': 62, '감사하다': 63, '많다': 64, '구입': 65, '조금': 66, '되어다': 67, '입다': 68, '반품': 69, '좋아하다': 70, '아쉽다': 71, 'ㅜㅜ': 72, '쓸다': 73, '아이': 74, '가다': 75, '깔끔하다': 76, '느낌': 77, '정도': 78, '아직': 79, '완전': 80, '불편하다': 81, '엄청': 82, '떨어지다': 83, '알다': 84, '나오다': 85, '넣다': 86, '처음': 87, '해주다': 88, '주다': 89, '사진': 90, '디자인': 91, '효과': 92, '튼튼하다': 93, '선물': 94, '대비': 95, '다시': 96, '상태': 97, 

<br>

### 2) Padding 적용

- 전체 훈련 데이터에서 각 샘플의 길이(or 문서 또는 문장의 단어의 수)는 서로 다를 수 있음  
- 이러한 샘플을 모델의 입력으로 사용하려면 **각 샘플의 길이를 동일하게 맞춰줘야 함** (이미지 데이터의 가로세로 크기를 맞춰주는 것과 동일)
- 이를 자연어 처리에서는 **패딩(Padding)**이라고 부름 
- 보통 **기준이 되는 길이**를 정하고, **숫자 0을 넣어서 길이가 다른 샘플들의 길이를 맞춰줌** 
<br>(정해준 길이보다 길이가 긴 샘플은 앞(or 뒤)의 데이터를 잘라내고, 정해준 길이보다 길이가 짧은 샘플은 뒤(or 앞)의 빈 자리를 0으로 채움)


In [14]:
len_nums = []

for idx, sublist in enumerate(train_sequences):
    len_nums.append([idx, len(sublist)])

print('longest sentence (index num & sequence length) :', sorted(len_nums, key=lambda x:x[1], reverse=True)[0], '\n')

print(train_sequences[130796])
print(len(train_sequences[130796]))

print(train_x.iloc[130796])
print(len(train_x.iloc[130796]))

longest sentence (index num & sequence length) : [130796, 44] 

[87, 12, 2, 4555, 375, 52, 111, 366, 311, 108, 2, 2339, 137, 108, 88, 2, 412, 6, 206, 1403, 67, 1295, 334, 580, 111, 1, 145, 580, 18, 137, 52, 13, 20, 69, 2, 106, 12, 510, 2, 449, 2, 182, 2, 2]
44
['처음', '구매', '하다', '옆쪽', '손잡이', '부분', '계속', '물이', '새다', '교환', '하다', '초기', '불량', '교환', '해주다', '하다', '그런데', '같다', '문제', '발생', '되어다', '톡톡', '문의', '답변', '계속', '박자', '늦다', '답변', '되다', '불량', '부분', '없다', '보다', '반품', '하다', '귀찮다', '구매', '확정', '하다', '검수', '하다', '판매', '하다', '하다']
44


In [15]:
MAX_LENGTH = 50 # >= longest sequence length 
# 위 MAX_LENGTH 값은 실제 적용할 텍스트데이터의 형태에 따라 증감합니다. 너무 큰 값을 지정할 경우 메모리가 부족할 수 있으며, 그러한 경우에는 길이가 긴 텍스트를 문장 단위 등으로 더 나눠 활용합니다.

train_inputs = pad_sequences(train_sequences, 
                             maxlen=MAX_LENGTH, 
                             padding='post') # truncating='pre' <- 길이 초과 시 default로 앞의 데이터를 잘라서 버림

test_inputs = pad_sequences(test_sequences, 
                             maxlen=MAX_LENGTH, 
                             padding='post') # truncating='pre' <- 길이 초과 시 default로 앞의 데이터를 잘라서 버림

print(train_inputs.shape)

(139935, 50)


In [16]:
print(train_sequences[0]) # original sequence (without padding)
print(train_x.iloc[0]) # original tokens

[10, 12, 46, 509]
['재다', '구매', '저렴하다', '따다']


In [17]:
train_inputs[0] # sequence with padding <- maxlen == 50 & padding == 'post'

# 각 형태소는 1:1로 매칭되며 1부터 시작하는 sequence number를 갖고 있음 (ex. '<OOV>': 1, '하다': 2, '좋다': 3, ...)
# 각 형태소에 매칭되는 sequence number로 채워진 sequence에 대하여 빈 자리를 "0"으로 채우는 padding을 진행하였음

# padding 과정에서 추가된 (아래 출력 결과의) "0"이라는 sequence number는 word_vocab에 포함되어 있지 않음
# 따라서 전체 단어 집합에 "0"이라는 sequence number에 해당하는 가상의 형태소가 포함된 것으로 가정해야 하므로,
# 위에서 vocab_size(어휘 목록의 크기) 변수의 값을 지정할 때 +2 계산을 적용하였음 (== [OOV] token & [Padding] token)

array([ 10,  12,  46, 509,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [18]:
len(train_inputs[0]) # >= longest sequence length

50

In [19]:
print(word_vocab)

{'<OOV>': 1, '하다': 2, '좋다': 3, '너무': 4, '배송': 5, '같다': 6, '있다': 7, '자다': 8, '먹다': 9, '재다': 10, '않다': 11, '구매': 12, '없다': 13, '이다': 14, '쓰다': 15, '오다': 16, '사용': 17, '되다': 18, '사다': 19, '보다': 20, '제품': 21, '빠르다': 22, '가격': 23, '그냥': 24, '생각': 25, '받다': 26, '주문': 27, '아니다': 28, '많이': 29, '별로': 30, '사이즈': 31, '들다': 32, 'ㅠㅠ': 33, '만족하다': 34, '맛있다': 35, '포장': 36, '크다': 37, '써다': 38, '편하다': 39, '작다': 40, '그렇다': 41, '맞다': 42, '상품': 43, '안되다': 44, '괜찮다': 45, '저렴하다': 46, '이쁘다': 47, '넘다': 48, '싸다': 49, '냄새': 50, '정말': 51, '부분': 52, '보내다': 53, '시키다': 54, '나다': 55, '버리다': 56, '예쁘다': 57, '하나': 58, '모르다': 59, 'ㅎㅎ': 60, '아주': 61, '진짜': 62, '감사하다': 63, '많다': 64, '구입': 65, '조금': 66, '되어다': 67, '입다': 68, '반품': 69, '좋아하다': 70, '아쉽다': 71, 'ㅜㅜ': 72, '쓸다': 73, '아이': 74, '가다': 75, '깔끔하다': 76, '느낌': 77, '정도': 78, '아직': 79, '완전': 80, '불편하다': 81, '엄청': 82, '떨어지다': 83, '알다': 84, '나오다': 85, '넣다': 86, '처음': 87, '해주다': 88, '주다': 89, '사진': 90, '디자인': 91, '효과': 92, '튼튼하다': 93, '선물': 94, '대비': 95, '다시': 96, '상태': 97, 

<br> 

### 3) 전처리 적용 후 데이터 중간 저장 @ Google Drive 

In [20]:
# 1) Save
data_path = '/gdrive/MyDrive/colab_data/temp_data/'

# 구글 드라이브 내 "data_path" 경로의 폴더가 존재하지 않을 시 새로이 생성
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
data_dict = {}
data_dict['train_inputs'] = train_inputs
data_dict['test_inputs'] = test_inputs
data_dict['train_y'] = train_y
data_dict['test_y'] = test_y

with open(data_path + 'data_dict.pkl', 'wb') as f:
    pickle.dump(data_dict, f)
    

# 2) Load
# data_path = '/gdrive/MyDrive/colab_data/temp_data/'

# with open(data_path + 'data_dict.pkl', 'rb') as f:
#     data_dict = pickle.load(f)

# train_inputs = data_dict['train_inputs']
# test_inputs = data_dict['test_inputs']
# train_y = data_dict['train_y'] 
# test_y = data_dict['test_y'] 

<br> 

### + 메모리 부족 시 현재 메모리 상의 변수 목록 확인 & 제거

In [21]:
# local_vars = list(locals().items())

# for var, obj in local_vars:
#     obj_size = sys.getsizeof(obj) # Return the size of object in bytes -> 1064 * 1064 bytes == 1 MB
#     if (obj_size > 1064 * 1064) and (not var.startswith('_')): 
#         print(var, int(obj_size / (1064 * 1064)), 'MB')

In [ ]:
# del reviews_df
# del train_x   
# del test_x   

# del flat_review_texts 
# del flat_review_sequences  

<br>
<br>

# 3. 모델링 - Neural Network (LSTM: Long short-term memory)

- (TF Official) Keras를 사용한 순환 신경망 (RNN) @ https://www.tensorflow.org/guide/keras/rnn?hl=ko

- (TF Official) RNN을 사용한 텍스트 분류 @ https://www.tensorflow.org/tutorials/text/text_classification_rnn?hl=ko#모델_만들기

- (TF Official) Keras를 사용한 Masking & Padding @ https://www.tensorflow.org/guide/keras/masking_and_padding?hl=en

- (국문 온라인 책) 순환 신경망(Recurrent Neural Network) @ https://wikidocs.net/48558


<br>

## 3-1. Using Dense Network

In [ ]:
# Training x : train_inputs
# Training y : train_y
# Test x : test_inputs
# Test y : test_y 

In [23]:
VOCAB_SIZE = vocab_size # num_all_token - num_rare_token + 2 
MAX_LENGTH = 50 # >= longest sequence length
NUM_CLASSES = len(set(train_y)) # number of classes

print(VOCAB_SIZE)
print(MAX_LENGTH)
print(NUM_CLASSES)

16475
50
2


In [24]:
model = models.Sequential()

model.add(layers.Embedding(input_dim=VOCAB_SIZE, output_dim=32, input_length=MAX_LENGTH)) # 3-dimensional cube image @ https://j.mp/2ZjrRno
# VOCAB_SIZE == 16475 / dimension of the embedding vector == 32 / MAX_LENGTH == 50
# - input (2D tensor) : (batch_size == # of sequences, input_length == 50) 
# - output (3D tensor) : (batch_size == # of sequences, input_length == 50, output_dim == 32)
# -> Embedding layer는 각 단어에 대해 "훈련 가능한" 하나의 벡터를 저장 (훈련 후 유사한 의미를 가진 단어는 종종 비슷한 벡터를 갖게 됨)
# ->> One-hot vector vs Embedding vector @ https://wikidocs.net/33520 + Vector embeddings @ https://j.mp/3kDOX0w
    
model.add(layers.GlobalAveragePooling1D()) # 다음 셀 예제 참고 & 공식문서 참고 @ https://j.mp/3kzXrWr
# - input (3D tensor) : (batch_size == # of sequences, input_length == 50, output_dim == 32)
# - output (2D tensor) : (batch_size == # of sequences, output_dim == 32) 
# -> 2번째 차원(sentence 당 sequence number 개수 50) 방향으로 평균을 계산하여 (None, 50, 32) -> (None, 32) 의 배열 생성
# ->> 이를 통해 추후 입력으로 사용되는 sentence에 포함된 sequence number의 개수가 달라지더라도 (ex. test data로 들어오는 문장의 단어 개수가 50이 아닐 때)
# ->>> 같은 크기(None, 32)의 벡터로 변환하여 통과시킬 수 있게 됨 (길이가 다른 입력을 다루기 위한 도구)

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.Adam(),
              metrics=[metrics.SparseCategoricalAccuracy()])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 32)            527200    
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 531,682
Trainable params: 531,682
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
x = np.array([ np.arange(12).reshape(3, 4), np.arange(12).reshape(3, 4)*2, np.arange(12).reshape(3, 4)*3 ])

print(x.shape)
x # batch_size (# of sequences), input_length (tokens per sentence), output_dim (embedding vector per token) 

(3, 3, 4)


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[ 0,  2,  4,  6],
        [ 8, 10, 12, 14],
        [16, 18, 20, 22]],

       [[ 0,  3,  6,  9],
        [12, 15, 18, 21],
        [24, 27, 30, 33]]])

In [26]:
x = tf.keras.layers.GlobalAveragePooling1D()(x)

print(x.shape)
x.numpy() # (batch_size, output_dim) <- n개였던 input_length 방향(위 x 변수의 내부 2차원 행렬에서 열 방향에 해당) 숫자들이 1개의 숫자로 평균내어짐

# ex. [0, 4, 8] -> 4, [1, 5, 9] -> 5, [2, 6, 10] -> 6, [3, 7, 11] -> 7

(3, 4)


array([[ 4,  5,  6,  7],
       [ 8, 10, 12, 14],
       [12, 15, 18, 21]])

In [27]:
checkpoint_path = '/gdrive/MyDrive/colab_data/temp_data/saved_models/'

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
callback_checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path + 'best_model.h5', 
                                                monitor='val_sparse_categorical_accuracy',
                                                save_best_only=True, 
                                                verbose=1)

history = model.fit(train_inputs, train_y, validation_split=0.2,
                    epochs=15, batch_size=100,
                    verbose=1, callbacks=[callback_checkpoint])

Epoch 1/15
1107/1120 [============================>.] - ETA: 0s - loss: 0.3584 - sparse_categorical_accuracy: 0.8495
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.88580, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_model.h5
1120/1120 [==============================] - 8s 4ms/step - loss: 0.3579 - sparse_categorical_accuracy: 0.8499 - val_loss: 0.2933 - val_sparse_categorical_accuracy: 0.8858
Epoch 2/15
1114/1120 [============================>.] - ETA: 0s - loss: 0.2753 - sparse_categorical_accuracy: 0.8942
Epoch 2: val_sparse_categorical_accuracy improved from 0.88580 to 0.88630, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_model.h5
1120/1120 [==============================] - 5s 4ms/step - loss: 0.2753 - sparse_categorical_accuracy: 0.8941 - val_loss: 0.2886 - val_sparse_categorical_accuracy: 0.8863
Epoch 3/15
1115/1120 [============================>.] - ETA: 0s - loss: 0.2615 - sparse_categorical_accuracy: 0.9001


In [28]:
best_model = models.load_model(checkpoint_path + 'best_model.h5')

pred = best_model.predict(test_inputs)
pred = tf.argmax(pred, axis=1)

print(accuracy_score(pred, test_y)) 

0.8869824754472846


<br>

## 3-2. Using Bidirectional LSTM Network

In [29]:
model = models.Sequential()

model.add(layers.Embedding(input_dim=VOCAB_SIZE, output_dim=32, input_length=MAX_LENGTH))

# layers.SimpleRNN(64)
# layers.GRU(64)
# layers.LSTM(64) 
# layers.Bidirectional(layers.SimpleRNN(64))
# layers.Bidirectional(layers.GRU(64))
model.add(layers.Bidirectional(layers.LSTM(64)))
# + (Official) tf.keras.layers.LSTM @ https://j.mp/3hTu9QK
# + (Official) tf.keras.layers.Bidirectional @ https://j.mp/3isoGzn
# + (국문) SimpleRNN vs LSTM vs Bidirectional(LSTM) @ https://j.mp/3BozJST & https://j.mp/2WujGje
# + (국문) BiLSTM으로 한국어 스팀 리뷰 감성 분류하기 @ https://j.mp/3jiVdII

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.Adam(),
              metrics=[metrics.SparseCategoricalAccuracy()])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 32)            527200    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 593,634
Trainable params: 593,634
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
checkpoint_path = '/gdrive/MyDrive/colab_data/temp_data/saved_models/'

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
callback_checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path + 'best_model.h5', 
                                                monitor='val_sparse_categorical_accuracy',
                                                save_best_only=True, 
                                                verbose=1)

history = model.fit(train_inputs, train_y, validation_split=0.2,
                    epochs=15, batch_size=100,
                    verbose=1, callbacks=[callback_checkpoint])

Epoch 1/15
1117/1120 [============================>.] - ETA: 0s - loss: 0.3194 - sparse_categorical_accuracy: 0.8682
Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.88698, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_model.h5
1120/1120 [==============================] - 17s 10ms/step - loss: 0.3194 - sparse_categorical_accuracy: 0.8682 - val_loss: 0.2838 - val_sparse_categorical_accuracy: 0.8870
Epoch 2/15
1117/1120 [============================>.] - ETA: 0s - loss: 0.2566 - sparse_categorical_accuracy: 0.9010
Epoch 2: val_sparse_categorical_accuracy improved from 0.88698 to 0.89027, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_model.h5
1120/1120 [==============================] - 11s 10ms/step - loss: 0.2566 - sparse_categorical_accuracy: 0.9010 - val_loss: 0.2826 - val_sparse_categorical_accuracy: 0.8903
Epoch 3/15
1120/1120 [==============================] - ETA: 0s - loss: 0.2260 - sparse_categorical_accuracy: 0.9

In [31]:
best_model = models.load_model(checkpoint_path + 'best_model.h5')

pred = best_model.predict(test_inputs)
pred = tf.argmax(pred, axis=1)

print(accuracy_score(pred, test_y))

0.8916178947192903


In [32]:
# model = models.Sequential()
# model.add(layers.Embedding(input_dim=VOCAB_SIZE, output_dim=32, input_length=MAX_LENGTH))
# model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
# model.add(layers.Bidirectional(layers.LSTM(32)))
# model.add(layers.Dropout(0.3))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.3))
# model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

# model = models.Sequential()
# model.add(layers.Embedding(input_dim=VOCAB_SIZE, output_dim=128, input_length=MAX_LENGTH))
# model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
# model.add(layers.Bidirectional(layers.LSTM(64)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

<br>
<br>

# + Model & Tokenizer Save/Load -> Predict on a new sentence

### 1) Save the model & tokenizer

In [33]:
from google.colab import drive
import pickle

drive.mount('/gdrive')

data_path = '/gdrive/MyDrive/colab_data/temp_data/'
if not os.path.exists(data_path):
    os.makedirs(data_path)


best_model.save(data_path + 'model_biLSTM_v1.h5') # Model 저장

with open(data_path + 'tokenizer-tf-16474.pkl', 'wb') as f:
    pickle.dump(tokenizer, f) # tensorflow.keras.preprocessing.text.Tokenizer 저장

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### 2) Load the model & tokenizer -> Predict on new sentences

In [34]:
from tensorflow.keras import models 
from google.colab import drive
import pickle

drive.mount('/gdrive')
data_path = '/gdrive/MyDrive/colab_data/temp_data/' 


saved_model = models.load_model(data_path + 'model_biLSTM_v1.h5') # Model 로딩 

with open(data_path + 'tokenizer-tf-16474.pkl', 'rb') as f:
    saved_tokenizer = pickle.load(f) # tensorflow.keras.preprocessing.text.Tokenizer 로딩

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [35]:
def tokenize_and_stemming(sentence, tokenizer):

    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence) 
    
    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming

    sentence_tokenized = []
    for token, pos in raw_pos_tagged: # ex) word <- ('서울', 'Noun')
        # pos 기준 제거 대상 : ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Suffix", "Determiner"]
        # 리뷰 텍스트 상 형태소의 순서를 고려하는 경우 의미를 갖게 됨 : "VerbPrefix" (잘/안/못), "Conjunction"(하지만/그러나/그리고), "KoreanParticle"(ㅋㅋㅋ/ㅎㅎㅎ/ㅠㅠ)
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)  

    return sentence_tokenized

### Predict on a single sentence

In [36]:
new_sentence = "아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요."


# 1) Tokenizing
okt = Okt()
tokenized_sentences = []
tokenized_sentences.append(tokenize_and_stemming(new_sentence, okt))

# 2) Tokens to sequence numbers
new_sequences = saved_tokenizer.texts_to_sequences(tokenized_sentences) 

# 3) Padding
new_inputs = pad_sequences(new_sequences, 
                             maxlen=50, # >= longest sequence length
                             padding='post') # truncating='pre' <- 길이 초과 시 default로 앞의 데이터를 잘라서 버림

# Prediction
print(saved_model.predict(new_inputs))
print(np.argmax(saved_model.predict(new_inputs), axis=1))

[[0.00564871 0.99435127]]
[1]


### Predict on multiple sentences

In [ ]:
new_sentences = ["아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.",
                "너무 맛있어요! 4명이 먹을거라 넉넉하게 샀다했는데 순식간에 없어집니다! 꼭 드셔보세요!", 
                "후기 좋아서 샀는데 진짜 별로에요... 재질두 비니루같은 재질....", 
                "내용물이 안나와요 ㅜㅜ 무용지물 ㅜㅜ 여러개 샀는데 ㅜㅜ"
                ]


# 1) Tokenizing
okt = Okt()
tokenized_sentences = []
for sentence in new_sentences:
    tokenized_sentences.append(tokenize_and_stemming(sentence, okt))

# 2) Tokens to sequence numbers
new_sequences = saved_tokenizer.texts_to_sequences(tokenized_sentences) 

# 3) Padding
new_inputs = pad_sequences(new_sequences, 
                             maxlen=50, # >= longest sequence length
                             padding='post') # truncating='pre' <- 길이 초과 시 default로 앞의 데이터를 잘라서 버림

# Prediction
print(saved_model.predict(new_inputs))
print(np.argmax(saved_model.predict(new_inputs), axis=1))

[[1.2652345e-02 9.8734760e-01]
 [1.3845420e-02 9.8615462e-01]
 [9.3328965e-01 6.6710316e-02]
 [9.9951184e-01 4.8820206e-04]]
[1 1 0 0]


### Classifier as a function

In [ ]:
def predict_sentiment(sentence, tokenizer, model):

    # 1) Tokenizing
    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence) 
    
    okt = Okt()
    raw_pos_tagged = okt.pos(sentence, stem=True)

    sentence_tokenized = []
    for token, pos in raw_pos_tagged: 
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)  

    tokenized_sentences = [sentence_tokenized]

    # 2) Tokens to sequence numbers
    new_sequences = saved_tokenizer.texts_to_sequences(tokenized_sentences) 

    # 3) Padding
    new_inputs = pad_sequences(new_sequences, maxlen=50, padding='post') 
    
    # Prediction
    prediction = model.predict(new_inputs)
    predicted_probability = np.round(np.max(prediction) * 100, 2) # ex) [[0.0125517 0.9874483]] -> round(0.9874483 * 100, 2) -> round(98.74483, 2) -> 98.74
    predicted_class = ['부정', '긍정'][np.argmax(prediction, axis=1)[0]] # ex) ['부정', '긍정'][np.argmax([0.0125517 0.9874483], axis=1)[0]] -> ['부정', '긍정'][[1][0]] -> ['부정', '긍정'][1] -> '긍정'
    
    print("{}% 확률로 {} 리뷰입니다.".format(predicted_probability, predicted_class))

In [ ]:
new_sentence = "아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요."

predict_sentiment(new_sentence, saved_tokenizer, saved_model)

98.73% 확률로 긍정 리뷰입니다.


In [ ]:
# "아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.",
# "너무 맛있어요! 4명이 먹을거라 넉넉하게 샀다했는데 순식간에 없어집니다! 꼭 드셔보세요!", 
# "후기 좋아서 샀는데 진짜 별로에요... 재질두 비니루같은 재질....", 
# "내용물이 안나와요 ㅜㅜ 무용지물 ㅜㅜ 여러개 샀는데 ㅜㅜ"

new_sentence = input() # 이거 살까말까 고민 많이 했는데 정작 써보니 나쁘지 않네요 곧 재구매 예정입니다!

predict_sentiment(new_sentence, saved_tokenizer, saved_model)

이거 살까말까 고민 많이 했는데 정작 써보니 나쁘지 않네요 곧 재구매 예정입니다!
81.63% 확률로 긍정 리뷰입니다.
